# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [83]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [84]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [85]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [86]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [87]:
# Getting the data from the top 10 expensive neighbourhoods
expensive_nei = to_data.reset_index().groupby("neighbourhood").mean()
expensive_nei = expensive_nei.sort_values("average_house_value", ascending=False).head(10)

# Calculate the mean number of dwelling types units per year
to_data_dwelling = to_data.drop(columns=['average_house_value','shelter_costs_owned','shelter_costs_rented']).mean().reset_index()

# Calculate the average monthly shelter costs for owned and rented dwellings
avg_shelter_costs = to_data[["shelter_costs_owned","shelter_costs_rented"]].groupby(to_data.index).mean()


### Panel Visualization Functions

In [88]:
# Define Panel visualization functions
def neighbourhood_map():
    nei_data = to_data.groupby("neighbourhood").mean()
    df_avg_location = pd.merge(df_neighbourhood_locations, nei_data, on="neighbourhood", how="inner")
    map_avg_nei = px.scatter_mapbox(
        df_avg_location,lat="lat",lon="lon",
        hover_data = df_avg_location,
        hover_name="neighbourhood",
        size="average_house_value",color="average_house_value",
        color_continuous_scale="icefire",
        title="Average House Value in Toronto",width=900, height=500)
    return map_avg_nei

def create_bar_chart(data, title, xlabel, ylabel, color):
    
    bar_chart = data.hvplot.bar(title=(title),xlabel=(xlabel), ylabel=(ylabel),color=(color), rot=45, width=500, height=500)
    return bar_chart
    

def create_line_chart(data, title, xlabel, ylabel, color):
    create_line_chart = data.hvplot.line(title=(title),xlabel=(xlabel), ylabel=(ylabel),color=(color),width=500, height=500)
    return create_line_chart


def top_most_expensive_neighbourhoods():
    average_house_value_snapshot = to_data.reset_index().groupby("neighbourhood").mean()
    top_most_expensive_neighbourhoods = average_house_value_snapshot.sort_values("average_house_value", ascending=False).head(10)
    fig = top_most_expensive_neighbourhoods.hvplot.bar(x='neighbourhood',  y='average_house_value',xlabel="Neighbourhood", ylabel="avg_house_value",title="top_most_expensive_neighbourhoods",rot=45,  width=500, height=500)
    return fig


In [89]:
# Line chart for owned dwellings
line_costs_owned = create_line_chart(data = avg_shelter_costs.iloc[:,0],title="Monthly Average Shelter Cost for Owned Dwellings in Toronto",xlabel="Year", ylabel="Avg Monthly Shelter Costs",color="blue",)
# Line chart for rented dwellings
line_costs_rented = create_line_chart(data = avg_shelter_costs.iloc[:,1],title="Monthly Average Shelter Cost for Rented Dwellings in Toronto",xlabel="Year", ylabel="Avg Monthly Shelter Costs",color="yellow",)


In [90]:
to_data_year = to_data.groupby('year')
to_data_year_sum = to_data_year.sum().drop(columns=['average_house_value', 'shelter_costs_owned', 'shelter_costs_rented'])

# number of dwelling types Bar chart for 2001
bar_chart_2001 = create_bar_chart(data = to_data_year_sum.iloc[0], title="Dwelling Types Units in Toronto in 2001", xlabel="2001", ylabel="Dwelling Type Units", color="red")

# number of dwelling types Bar chart for 2006
bar_chart_2006 = create_bar_chart(data = to_data_year_sum.iloc[1], title="Dwelling Types Units in Toronto in 2006", xlabel="2006", ylabel="Dwelling Type Units", color="blue")

# number of dwelling types Bar chart for 2011
bar_chart_2011 = create_bar_chart(data = to_data_year_sum.iloc[2], title="Dwelling Types Units in Toronto in 2011", xlabel="2011", ylabel="Dwelling Type Units", color="yellow")

# number of dwelling types Bar chart for 2016
bar_chart_2016 = create_bar_chart(data = to_data_year_sum.iloc[3], title="Dwelling Types Units in Toronto in 2016", xlabel="2016", ylabel="Dwelling Type Units", color="pink")

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [91]:
from panel.interact import interact
from panel import widgets

# Create a Title for the Dashboard
dashboard_title = 'Toronto Dwellings Analysis Dashboard'

# Define a welcome text
dashboard_welcome = 'Welcome to explore the Toronto Dwellings Analysis Dashboard'


# Create a tab layout for the dashboard

dashboard_column_1 = pn.Column( bar_chart_2001,bar_chart_2006,bar_chart_2011,bar_chart_2016)
dashboard_column_2 = pn.Column(line_costs_owned,line_costs_rented)
dashboard_column_3 = pn.Column(neighbourhood_map,top_most_expensive_neighbourhoods)

# Create the main dashboard

dashboard = pn.Column(dashboard_title, pn.Tabs(('Welcome & Enjoy',dashboard_welcome),('## Overview of the market', dashboard_column_1), ('## Housing Cost', dashboard_column_2),('## Map view and Best neighbourhood',dashboard_column_3)))

## Serve the Panel Dashboard

In [92]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Markdown(str, name='Welcome & Enjoy')
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Bars)
            [2] HoloViews(Bars)
            [3] HoloViews(Bars)
        [2] Column
            [0] HoloViews(Curve)
            [1] HoloViews(Curve)
        [3] Column
            [0] Column
                [0] Column()
                [1] Row
                    [0] Plotly(Figure, name='interactive06254')
            [1] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive06260')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [93]:
# neighbourhood_map().show()

In [94]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [95]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [96]:
# average_house_value()

In [97]:
# average_value_by_neighbourhood()

In [98]:
# number_dwelling_types()

In [99]:
# average_house_value_snapshot()

In [100]:
# top_most_expensive_neighbourhoods()

In [101]:
# sunburts_cost_analysis()